In [1]:
# ==============================================================================
# SEL 1: INSTALASI DAN LOGIN
# Jalankan sel ini terlebih dahulu.
# ==============================================================================

# 1. Instal semua library yang diperlukan dalam mode senyap (-q)
#print("🚀 Menginstal library yang dibutuhkan...")
#!pip install -q transformers peft accelerate bitsandbytes trl datasets
#print("✅ Library berhasil diinstal.")

# 2. Login ke Hugging Face Hub
# Anda akan diminta memasukkan token. Dapatkan dari: https://huggingface.co/settings/tokens
import huggingface_hub
print("\nSilakan login ke Hugging Face Hub:")
huggingface_hub.login()

🚀 Menginstal library yang dibutuhkan...
✅ Library berhasil diinstal.

Silakan login ke Hugging Face Hub:


In [2]:
# ==============================================================================
# SEL 2: SCRIPT LENGKAP FINE-TUNING DAN INFERENSI
# Jalankan sel ini setelah setup selesai.
# ==============================================================================

# 3. Impor semua library yang akan digunakan di sel berikutnya

import os
import torch
from datasets import Dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    pipeline,
)
from peft import LoraConfig, PeftModel, get_peft_model
from trl import SFTTrainer
import json

In [3]:
# ==============================================================================
# SEL 3: SCRIPT UTAMA (FINE-TUNING & INFERENSI)
# Jalankan sel ini setelah SEL 2 selesai.
# ==============================================================================

class InstructionTuningPipeline:
    """
    Sebuah kelas untuk merangkum seluruh proses fine-tuning dan inferensi.
    """
    def __init__(self, base_model_name: str, new_model_name: str):
        self.base_model_name = base_model_name
        self.new_model_name = new_model_name
        self.tokenizer = None
        self.model = None

    def _create_prompt_format(self, sample):
        return f"<s>[INST] {sample['prompt']} [/INST] {sample['output']} </s>"

    def _load_model_and_tokenizer(self):
        print("🚀 Memuat model dasar dan tokenizer dengan kuantisasi 4-bit...")
        bnb_config = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_quant_type="nf4",
            bnb_4bit_compute_dtype=torch.bfloat16,
            bnb_4bit_use_double_quant=True,
        )
        self.model = AutoModelForCausalLM.from_pretrained(
            self.base_model_name,
            quantization_config=bnb_config,
            device_map="auto"
        )
        self.model.config.use_cache = False
        self.model.config.pretraining_tp = 1

        self.tokenizer = AutoTokenizer.from_pretrained(self.base_model_name, trust_remote_code=True)
        self.tokenizer.pad_token = self.tokenizer.eos_token
        self.tokenizer.padding_side = "right"
        print("✅ Model dan tokenizer berhasil dimuat.")

    def run_finetuning(self):
        self._load_model_and_tokenizer()

        peft_config = LoraConfig(
            lora_alpha=16,
            lora_dropout=0.1,
            r=32,
            bias="none",
            task_type="CAUSAL_LM",
            target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj"]
        )
        self.model = get_peft_model(self.model, peft_config)

        dummy_data = [
            {"prompt": "How do I reset my password in the e-commerce app?", "output": "1. Open the app\n2. Go to the login page\n3. Tap on the 'Forgot Password' link\n4. Enter your registered email address\n5. Tap 'Submit' to request a reset link\n6. Check your email and verify the reset request\n7. Follow the link to set a new password"},
            {"prompt": "How to update my profile picture?", "output": "1. Go to your profile page.\n2. Tap on your current picture.\n3. Select 'Upload a new photo'.\n4. Choose a photo from your gallery.\n5. Crop and adjust the photo.\n6. Tap 'Save'."}
        ]
        dataset = Dataset.from_list(dummy_data)

        training_arguments = TrainingArguments(
            output_dir="./results",
            num_train_epochs=1,
            per_device_train_batch_size=1,
            gradient_accumulation_steps=4,
            optim="paged_adamw_32bit",
            save_steps=50,
            logging_steps=10,
            learning_rate=2e-4,
            weight_decay=0.001,
            fp16=True,
            max_grad_norm=0.3,
            max_steps=-1,
            warmup_ratio=0.03,
            group_by_length=True,
            lr_scheduler_type="cosine",
        )

        trainer = SFTTrainer(
            model=self.model,
            train_dataset=dataset,
            peft_config=peft_config,
            formatting_func=self._create_prompt_format,
            # max_seq_length=512, # Baris ini dihapus untuk mengatasi error TypeError
            #tokenizer=self.tokenizer,
            args=training_arguments,
        )

        print("\n🚀 Memulai proses fine-tuning... Ini akan memakan waktu beberapa menit.")
        trainer.train()
        print("✅ Proses fine-tuning selesai.")

        trainer.model.save_pretrained(self.new_model_name)
        self.tokenizer.save_pretrained(self.new_model_name)
        print(f"✅ Adapter model berhasil disimpan di '{self.new_model_name}'")

    def generate_instructions(self, user_prompt: str):
        print("\n🚀 Menjalankan inferensi dengan model yang sudah di-fine-tune...")

        base_model = AutoModelForCausalLM.from_pretrained(
            self.base_model_name,
            low_cpu_mem_usage=True,
            return_dict=True,
            torch_dtype=torch.float16,
            device_map="auto",
        )

        try:
            model = PeftModel.from_pretrained(base_model, self.new_model_name)
            model = model.merge_and_unload()
            print("✅ Adapter berhasil digabungkan dengan model dasar.")
        except Exception as e:
            print(f"⚠️ Gagal memuat adapter: {e}. Menggunakan model dasar untuk inferensi.")
            model = base_model

        tokenizer = AutoTokenizer.from_pretrained(self.base_model_name, trust_remote_code=True)
        tokenizer.pad_token = tokenizer.eos_token

        generator = pipeline("text-generation", model=model, tokenizer=tokenizer)
        formatted_prompt = f"<s>[INST] {user_prompt} [/INST]"

        sequences = generator(
            formatted_prompt,
            max_new_tokens=256,
            do_sample=True,
            temperature=0.7,
            top_k=50,
            top_p=0.95,
            num_return_sequences=1,
        )

        result = sequences[0]['generated_text']
        answer = result.split("[/INST]")[1].strip()

        print("\n--- HASIL GENERASI ---")
        print(answer)
        print("----------------------\n")
        return answer

def illustrate_api_integration(prompt: str, generated_instructions: str):
    print("✨ Ilustrasi Integrasi API ✨")
    api_response = {
        "status": "success",
        "prompt": prompt,
        "instructions": generated_instructions
    }
    print(json.dumps(api_response, indent=2, ensure_ascii=False))
    print("---------------------------\n")




In [ ]:
# --- Konfigurasi dan Eksekusi (Versi Lebih Hemat RAM) ---
if __name__ == "__main__":
    BASE_MODEL = "mistralai/Mistral-7B-Instruct-v0.2"
    ADAPTER_MODEL = "mistral-7b-task-instructions-adapter"

    pipeline_instance = InstructionTuningPipeline(
        base_model_name=BASE_MODEL,
        new_model_name=ADAPTER_MODEL
    )

    # Langkah 1: Jalankan fine-tuning
    # Modifikasi: Fungsi run_finetuning sekarang akan mengembalikan model yang sudah dilatih
    trained_model, tokenizer = pipeline_instance.run_finetuning()

    # Langkah 2: Jalankan inferensi
    # Modifikasi: Berikan model yang sudah dilatih ke fungsi inferensi agar tidak perlu load ulang
    user_prompt = "Bagaimana cara mereset password di aplikasi e-commerce 'xx'?"
    generated_text = pipeline_instance.generate_instructions(user_prompt, model=trained_model, tokenizer=tokenizer)

    # Langkah 3: Ilustrasikan integrasi
    illustrate_api_integration(prompt=user_prompt, generated_instructions=generated_text)

🚀 Memuat model dasar dan tokenizer dengan kuantisasi 4-bit...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

✅ Model dan tokenizer berhasil dimuat.


/usr/local/lib/python3.12/dist-packages/peft/tuners/lora/bnb.py:348: UserWarning: Merge lora module to 4-bit linear may get different generations due to rounding errors.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/peft/tuners/tuners_utils.py:196: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(
